# CIFAR-10: Adversarial Training and Defenses

## Imports and CIFAR-10 loading

In [1]:
# For plotting
import numpy as np
import torch
import torch.nn as nn

# Nice loading bars
from tqdm.notebook import tnrange, tqdm

# DNN used
import models.resnet as resnet

# Test the loaded model
import utils.clean_test as clean_test

In [2]:
# Define the `device` PyTorch will be running on, please hope it is CUDA
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Notebook will use PyTorch Device: " + device.upper())

Notebook will use PyTorch Device: CUDA


## Clean Datasets

In [3]:
# Get the data loaders (assume we do no validation)
import utils.dataloaders as dataloaders

DATA_ROOT = "./datasets/"

trainSetLoader, _, testSetLoader = dataloaders.get_CIFAR10_data_loaders(
    DATA_ROOT,
    trainSetSize=50000,
    validationSetSize=0,
    batchSize=128,
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## Adversarial Training

In [4]:
import attacks.fgsm as fgsm
import attacks.pgd as pgd

attacks = {}
attacks["FGSM"] = fgsm.fgsm_attack
attacks["PGD"] = pgd.pgd_attack

import utils.attacking as attacking

# For printing outcomes
import utils.printing as printing

In [5]:
# Adversarial examples should be typically generated when model parameters are not
# changing i.e. model parameters are frozen. This step may not be required for very
# simple linear models, but is a must for models using components such as dropout
# or batch normalization.


# Note: to speed up training, using this https://arxiv.org/abs/2001.03994 variant
def get_adversarially_trained_model(attack, **kwargs):
    # Helps adjust learning rate for better results
    def adjust_learning_rate(optimizer, epoch, learning_rate):
        actual_learning_rate = learning_rate
        if epoch >= 100:
            actual_learning_rate = 0.01
        if epoch >= 150:
            actual_learning_rate = 0.001
        for param_group in optimizer.param_groups:
            param_group["lr"] = actual_learning_rate

    # Various training parameters
    epochs = 200
    learning_rate = 0.05

    # Network parameters
    loss_function = nn.CrossEntropyLoss()
    model = resnet.ResNet18()
    model = model.to(device)
    model = nn.DataParallel(model)

    # Consider using ADAM here as another gradient descent algorithm
    optimizer = torch.optim.SGD(
        model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002
    )

    # Check if using epsilon
    if "epsilon" in kwargs:
        epsilon = kwargs["epsilon"]
    else:
        epsilon = None

    # Check if using alpha
    if "alpha" in kwargs:
        alpha = kwargs["alpha"]
    else:
        alpha = None

    # Get iterations
    if "iterations" in kwargs:
        iterations = kwargs["iterations"]
    else:
        iterations = None

    # Get the attack
    attack_function = attacks[attack]

    print("Training the model using adversarial examples...")
    model.train()

    # Use a pretty progress bar to show updates
    for epoch in tnrange(epochs, desc="Adversarial Training Progress"):
        total_epoch_loss = 0

        for _, (images, labels) in enumerate(tqdm(trainSetLoader, desc="Batches")):
            # Cast to proper tensors
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            # Run the attack
            model.eval()
            perturbed_images = attack_function(
                images,
                labels,
                model,
                nn.CrossEntropyLoss(),
                epsilon=epsilon,
                alpha=alpha,
                scale=True,
                iterations=iterations,
            )
            model.train()

            # Predict and optimise
            logits = model(perturbed_images)
            loss = loss_function(logits, labels)

            # Early stopping if loss becomes nan
            if np.isnan(loss.item()):
                print("...terminating early due to loss being nan...")

                return model

            # Track total loss
            total_epoch_loss += loss.item()

            # Gradient descent
            loss.backward()

            # Also clip the gradients (for some reason they super explode and loss becomes nan)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()

        # To track if the model is getting better
        print("Loss is ", total_epoch_loss)
        model.eval()
        clean_test.test_trained_model(model, testSetLoader)
        attacking.attack_model(
            model,
            testSetLoader,
            "FGSM",
            attacks["FGSM"],
            epsilon=epsilon,
        )
        model.train()

    print("... done!")

    # Return the trained model
    return model

In [6]:
# Quite similar to the one above, but this one introduces IAT (Interpolated Adversarial Training)
import defenses.iat as iat


def get_interpolated_adversarially_trained_model(attack, **kwargs):
    # Helps adjust learning rate for better results
    def adjust_learning_rate(optimizer, epoch, learning_rate):
        actual_learning_rate = learning_rate
        if epoch >= 100:
            actual_learning_rate = 0.01
        if epoch >= 150:
            actual_learning_rate = 0.001
        for param_group in optimizer.param_groups:
            param_group["lr"] = actual_learning_rate

    # Various training parameters
    epochs = 200
    learning_rate = 0.05

    # Network parameters
    loss_function = nn.CrossEntropyLoss()
    model = resnet.ResNet18()
    model = model.to(device)
    model = nn.DataParallel(model)

    # Consider using ADAM here as another gradient descent algorithm
    optimizer = torch.optim.SGD(
        model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002
    )

    # Check if using epsilon
    if "epsilon" in kwargs:
        epsilon = kwargs["epsilon"]
    else:
        epsilon = None

    # Check if using alpha
    if "alpha" in kwargs:
        alpha = kwargs["alpha"]
    else:
        alpha = None

    # Get iterations
    if "iterations" in kwargs:
        iterations = kwargs["iterations"]
    else:
        iterations = None

    # Get the attack
    attack_function = attacks[attack]

    print("Training the model using adversarial examples...")
    model.train()

    # Use a pretty progress bar to show updates
    for epoch in tnrange(epochs, desc="Adversarial Training Progress"):
        for _, (images, labels) in enumerate(tqdm(trainSetLoader, desc="Batches")):
            # Cast to proper tensors
            images, labels = images.to(device), labels.to(device)

            # Make sure previous step gradients are not used
            optimizer.zero_grad()

            # Use manifold mixup to modify the data
            (
                benign_mix_images,
                benign_mix_labels_a,
                benign_mix_labels_b,
                benign_mix_lamda,
            ) = iat.mix_inputs(1, images, labels)

            # Predict and calculate benign loss
            benign_logits = model(benign_mix_images)
            benign_loss = iat.mixup_loss_function(
                loss_function,
                benign_mix_lamda,
                benign_logits,
                benign_mix_labels_a,
                benign_mix_labels_b,
            )

            # Run the adversarial attack
            model.eval()
            perturbed_images = attack_function(
                images,
                labels,
                model,
                loss_function,
                epsilon=epsilon,
                alpha=alpha,
                scale=True,
                iterations=iterations,
            )
            model.train()

            # Use manifold mixup on the adversarial data
            (
                adversarial_mix_images,
                adversarial_mix_labels_a,
                adversarial_mix_labels_b,
                adversarial_mix_lamda,
            ) = iat.mix_inputs(1, perturbed_images, labels)

            # Predict and calculate adversarial loss
            adversarial_logits = model(adversarial_mix_images)
            adversarial_loss = iat.mixup_loss_function(
                loss_function,
                adversarial_mix_lamda,
                adversarial_logits,
                adversarial_mix_labels_a,
                adversarial_mix_labels_b,
            )

            # Take average of the two losses
            loss = (benign_loss + adversarial_loss) / 2

            # Gradient descent
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 10)
            optimizer.step()

        # To track if the model is getting better
        model.eval()
        clean_test.test_trained_model(model, testSetLoader)
        attacking.attack_model(
            model,
            testSetLoader,
            "FGSM",
            attacks["FGSM"],
            epsilon=epsilon,
        )
        model.train()

    print("... done!")

    # Return the trained model
    return model

### FGSM Adversarial Training

In [ ]:
fgsm_model = get_adversarially_trained_model("FGSM", epsilon=0.2)

# From this point on, we simply want to evaluate
fgsm_model.eval()
clean_test.test_trained_model(fgsm_model, testSetLoader)

In [ ]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [ ]:
for epsilon in epsilons:
    attacking.attack_model(
        fgsm_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

In [ ]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 4 / 255, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [ ]:
for epsilon in epsilons:
    attacking.attack_model(
        fgsm_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=20,
    )

In [ ]:
# Make sure to test the final accuracy of the model
clean_test.test_trained_model(fgsm_model, testSetLoader)

In [ ]:
# Save the FGSM model
torch.save(fgsm_model, "./cifar10_trained_fgsm_model")

### PGD Adversarial Training

In [ ]:
pgd_model = get_adversarially_trained_model(
    "PGD", epsilon=(8 / 255), alpha=(2 / 255), iterations=7
)

In [ ]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [ ]:
for epsilon in epsilons:
    attacking.attack_model(
        pgd_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

In [ ]:
# Several values to use for the epsilons
epsilons = [0, 4 / 255, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [ ]:
for epsilon in epsilons:
    attacking.attack_model(
        pgd_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=7,
    )

In [ ]:
# Make sure to test the final accuracy of the model
clean_test.test_trained_model(pgd_model, testSetLoader)

## Loading a good (saved) model

In [19]:
pgd_model = torch.load("./cifar10_pgd_model")
pgd_model.eval()

clean_test.test_trained_model(pgd_model, testSetLoader)

Testing the model...


Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 83.53%


In [20]:
# Several values to use for the epsilons
epsilons = [0, 4 / 255, 8 / 255, 0.05, 0.1, 0.25]

In [21]:
for epsilon in epsilons:
    attacking.attack_model(
        pgd_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

Testing the model under FGSM Attack using epsilon = 0, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 83.53%


Testing the model under FGSM Attack using epsilon = 0.01568627450980392, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 66.77%


Testing the model under FGSM Attack using epsilon = 0.03137254901960784, alpha = None...


...

... done! Accuracy: 38.91%


Testing the model under FGSM Attack using epsilon = 0.1, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 22.83%


Testing the model under FGSM Attack using epsilon = 0.25, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 18.9%




In [22]:
# Several values to use for the epsilons
epsilons = [0, 4 / 255, 8 / 225, 0.05, 0.1, 0.25]

In [23]:
for epsilon in epsilons:
    attacking.attack_model(
        pgd_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=7,
    )

Testing the model under PGD Attack using epsilon = 0, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 83.53%


Testing the model under PGD Attack using epsilon = 0.01568627450980392, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 64.32%


Testing the model under PGD Attack using epsilon = 0.035555555555555556, alpha = 0.00784313725490196...


...

... done! Accuracy: 37.18%


Testing the model under PGD Attack using epsilon = 0.1, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 27.12%


Testing the model under PGD Attack using epsilon = 0.25, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 8.66%




In [24]:
attacking.attack_model(
    pgd_model,
    testSetLoader,
    "PGD",
    attacks["PGD"],
    epsilon=(4 / 255),
    alpha=(2 / 255),
    iterations=20,
)

Testing the model under PGD Attack using epsilon = 0.01568627450980392, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 64.14%




## Interpolated Adversarial Training (IAT)

In [12]:
# iat_pgd_model = get_interpolated_adversarially_trained_model(
#     "PGD", epsilon=(8 / 255), alpha=(2 / 255), iterations=7
# )

In [13]:
iat_pgd_model = torch.load('cifar_trained_iat_model')

In [14]:
# Several values to use for the epsilons
epsilons = [0, 4 / 255, 8 / 255, 0.05, 0.1, 0.25]

In [15]:
for epsilon in epsilons:
    attacking.attack_model(
        iat_pgd_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

Testing the model under FGSM Attack using epsilon = 0, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 92.1%


Testing the model under FGSM Attack using epsilon = 0.01568627450980392, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 66.22%


Testing the model under FGSM Attack using epsilon = 0.03137254901960784, alpha = None...


...

... done! Accuracy: 49.72%


Testing the model under FGSM Attack using epsilon = 0.1, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 42.28%


Testing the model under FGSM Attack using epsilon = 0.25, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 33.03%




In [16]:
# Several values to use for the epsilons
epsilons = [0, 4 / 255, 8 / 225, 0.05, 0.1, 0.25]

In [17]:
for epsilon in epsilons:
    attacking.attack_model(
        iat_pgd_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=7,
    )

Testing the model under PGD Attack using epsilon = 0, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 92.1%


Testing the model under PGD Attack using epsilon = 0.01568627450980392, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 44.84%


Testing the model under PGD Attack using epsilon = 0.035555555555555556, alpha = 0.00784313725490196...


...

... done! Accuracy: 30.97%


Testing the model under PGD Attack using epsilon = 0.1, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 26.38%


Testing the model under PGD Attack using epsilon = 0.25, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 18.57%




In [18]:
attacking.attack_model(
    iat_pgd_model,
    testSetLoader,
    "PGD",
    attacks["PGD"],
    epsilon=(4 / 255),
    alpha=(2 / 255),
    iterations=20,
)

Testing the model under PGD Attack using epsilon = 0.01568627450980392, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 43.65%


